In [1]:
# Assuming douban2fb.txt is a mapping file with movie ID to entity ID
mapping_file = 'douban2fb.txt'

# Create a set to store the entity IDs from douban2fb.txt for faster lookup
entity_ids_list = []
movie_id2fb = {}

with open(mapping_file, 'r') as mapping:
    for line in mapping:
        # Assuming the file format is "movieID\tentityID"
        movie_id, entity_id = line.strip().split()
        entity_id="<http://rdf.freebase.com/ns/" + entity_id+ ">"
        movie_id2fb[movie_id] = entity_id
        entity_ids_list.append(entity_id)

In [3]:
import gzip
from tqdm import tqdm
str_flag="<http://rdf.freebase.com/ns/"
str_fault="<http://rdf.freebase.com/ns/people.person.date_of_birth>"
with gzip.open('freebase_douban.gz', 'rb') as f, gzip.open('output_first_entity.gz', 'wt') as output1, gzip.open('output_first_3.gz', 'wt') as output2:
    for line in tqdm(f, desc="Processing lines", unit="line"):
        triplet = line.strip().decode().split('\t')[:3]
        #头实体在entity_ids_list中
        if triplet[0] in entity_ids_list and str_flag in triplet[2] and str_fault not in triplet[1]:
            output1.write(triplet[2]+"\n")
            output2.write(triplet[0]+"\t"+triplet[1]+"\t"+triplet[2]+"\n")


Processing lines: 395577070line [46:22, 142186.26line/s]


In [2]:
import gzip
from tqdm import tqdm
first_3 = []#存储第一跳的三元组
first_jump = []#存储第一跳的实体
#将output_first_entity.gz与output_first_3.gz中的实体和三元组存入字典
with gzip.open('output_first_entity.gz', 'rb') as f:
    for line in tqdm(f, desc="Processing lines", unit="line"):
        entity = line.strip().decode()
        first_jump.append(entity)
with gzip.open('output_first_3.gz','rb') as f:
    for line in tqdm(f,desc="Processing lines",unit="line"):
        triplet = line.strip().decode().split("\t")[:3]
        first_3.append(triplet)

print(first_3[4])
print(first_jump[4])

Processing lines: 0line [00:00, ?line/s]

Processing lines: 125372line [00:00, 918191.52line/s]
Processing lines: 125372line [00:00, 456479.39line/s]

['<http://rdf.freebase.com/ns/m.012x63>', '<http://rdf.freebase.com/ns/film.film.dubbing_performances>', '<http://rdf.freebase.com/ns/m.0p7zxhh>']
<http://rdf.freebase.com/ns/m.0p7zxhh>


In [3]:
print(first_3[4][0])

<http://rdf.freebase.com/ns/m.012x63>


In [4]:
import copy
#关系<=50则删除，实体<=20则删除
min_relation = 50
min_entity = 20

#函数，通过关系小于50更新first_3，通过first_3更新first_jump
def update_first_3_by_relation (first_3,min_relation,first_jump):
    num_relation = {}#存储每个关系的数量
    num_of_entity = len(first_3)#实体的数量
    for i in range(num_of_entity):
        if first_3[i][1] not in num_relation:
            num_relation[first_3[i][1]] = 1
        else:
            num_relation[first_3[i][1]] = num_relation[first_3[i][1]] + 1
    id_to_delete = []#存储需要删除的三元组的id
    #在first_3中删除关系小于50的三元组,并在first_jump中删除对应的实体
    for i in range(num_of_entity):
        if num_relation[first_3[i][1]] <= min_relation and first_jump[i] not in entity_ids_list:
            id_to_delete.append(i)
    for i in range(len(id_to_delete)):
        del first_3[id_to_delete[i]-i]
        del first_jump[id_to_delete[i]-i]
    
def update_first_3_by_entity(first_3,min_entity,first_jump):
    num_entity = {}#存储每个实体的数量
    num_of_entity = len(first_3)#实体的数量,即三元组的数量
    for i in range(num_of_entity):
        if first_jump[i] not in num_entity:
            num_entity[first_jump[i]] = 1
        else:
            num_entity[first_jump[i]] = num_entity[first_jump[i]] + 1
    id_to_delete = []#存储需要删除的三元组的id
    #在first_3中删除实体小于20且不在entity_id_list的三元组,并在first_jump中删除对应的实体
    for i in range(num_of_entity):
        if num_entity[first_jump[i]] <= min_entity and first_jump[i] not in entity_ids_list:
            id_to_delete.append(i)
    for i in range(len(id_to_delete)):
        del first_3[id_to_delete[i]-i]
        del first_jump[id_to_delete[i]-i]
            
#处理第一跳，删除关系小于50的三元组，删除实体小于20的三元组,轮流执行直至first_3和first_jump不再变化
def process_first_jump(first_3,first_jump,min_entity,min_realtion):
    first_3_old = copy.deepcopy(first_3)
    first_jump_old = copy.deepcopy(first_jump)
    update_first_3_by_relation(first_3,min_relation,first_jump)
    update_first_3_by_entity(first_3,min_entity,first_jump)
    while first_3_old != first_3 and first_jump_old != first_jump:
        first_3_old = copy.deepcopy(first_3)
        first_jump_old = copy.deepcopy(first_jump)
        update_first_3_by_relation(first_3,min_relation,first_jump)
        update_first_3_by_entity(first_3,min_entity,first_jump)
    
process_first_jump(first_3,first_jump,min_entity,min_relation)

In [5]:
#测试代码
#检测first_3和first_jump是否一一对应
print(len(first_3))
print(len(first_jump))
for i in range(len(first_3)):
    if first_3[i][0] == first_jump[i] and first_3[i][2] in entity_ids_list:
        continue
    elif first_3[i][2] == first_jump[i] and first_3[i][0] in entity_ids_list:
        continue
    else:
        print("error")
        print(i)
        print(first_3[i])
        print(first_jump[i])
        
        

24669
24669


In [6]:
#测试代码
#测试正确性
num_entity_first_jump = {}
num_relation_first_jump = {}
num_of_entity = len(first_jump)
for i in range(num_of_entity):
    if first_jump[i] not in num_entity_first_jump:
        num_entity_first_jump[first_jump[i]] = 1
    else:
        num_entity_first_jump[first_jump[i]] = num_entity_first_jump[first_jump[i]] + 1
    if first_3[i][1] not in num_relation_first_jump:
        num_relation_first_jump[first_3[i][1]] = 1
    else:
        num_relation_first_jump[first_3[i][1]] = num_relation_first_jump[first_3[i][1]] + 1

#测试正确性,测试数量是否符合要求
min_relation = 50
min_entity = 20
for i in range(num_of_entity):
    if num_entity_first_jump[first_jump[i]] < min_entity and first_jump[i] not in entity_ids_list:
        print("error")
        print(i)
        print(first_jump[i])
    if num_relation_first_jump[first_3[i][1]] < min_relation and first_jump[i] not in entity_ids_list:
        print("error")
        print(i)
        print(first_3[i][1])

print(len(num_entity_first_jump))
print(len(num_relation_first_jump))


281
20


In [10]:
#判断first_3中的实体是否包含entity_ids_list中的实体
entity_in_first_3 = []
for i in range(len(first_3)):
    entity_in_first_3.append(first_3[i][0])
    entity_in_first_3.append(first_3[i][2])
entity_in_first_3 = list(set(entity_in_first_3))
print(len(entity_in_first_3))
for i in range(len(entity_ids_list)):
    if entity_ids_list[i] not in entity_in_first_3:
        print("error")
        print(entity_ids_list[i])

763


In [11]:
#将first_jump与entity_id_list合并
first_jump_final = first_jump + entity_ids_list
#第一跳收尾，将first_jump变成不含重复实体的列表
first_jump_final = list(set(first_jump_final))
print(len(first_jump_final))
#将first_jump写入文件
with gzip.open('first_jump.gz','wb') as f:
    for i in first_jump_final:
        f.write((i+"\n").encode())
#将first_3写入文件
with gzip.open('first_3.gz','wb') as f:
    for i in range(len(first_3)):
        f.write((first_3[i][0]+"\t"+first_3[i][1]+"\t"+first_3[i][2]+"\n").encode())

763
